<h1>Introdução</h1>
Uma Rede Neural Artificial (RNA) é um modelo computacional inspirado na dinâmica funcional do cérebro humano. Assim como os neurônios biológicos, as redes neurais artificiais são compostas por unidades chamadas neurônios artificiais, que recebem entradas, realizam cálculos matemáticos e produzem uma saída. Quando o valor calculado ultrapassa determinado limite (threshold), o neurônio é ativado.

Essas redes são formadas por camadas de neurônios interconectados, que processam informações de maneira distribuída. A partir das combinações de entradas e dos ajustes dos pesos de cada conexão, as RNAs conseguem aprender padrões complexos e realizar tarefas de previsão e classificação.

As redes neurais artificiais são capazes de resolver uma ampla variedade de problemas, como reconhecimento de caracteres manuscritos, detecção facial, processamento de linguagem natural e muitas outras aplicações no campo do Deep Learning.

Apesar de seu enorme sucesso prático, a maioria das RNAs funciona como uma “caixa-preta”: sabemos que elas aprendem a resolver os problemas de forma eficaz, mas analisar em detalhe como cada decisão interna é tomada ainda é um desafio.


# Perceptrons

O **perceptron** é o modelo mais simples de rede neural artificial, representando de forma abstrata um único neurônio. Ele recebe um conjunto de entradas (geralmente valores binários), atribui pesos a cada uma delas, soma esses valores e aplica uma **função de ativação** para decidir se deve “disparar” ou não.

Matematicamente, para entradas $x_1, x_2, \dots, x_n$ com respectivos pesos $w_1, w_2, \dots, w_n$ e um termo de ajuste chamado **bias** ($b$), o perceptron calcula:

$$
z = \sum_{i=1}^{n} w_i x_i + b
$$

A saída é então determinada pela função de ativação (no perceptron clássico, uma função **degrau**):

$$
y = f(z) =
\begin{cases}
1 & \text{se } z \geq 0 \\
0 & \text{se } z < 0
\end{cases}
$$

Isso significa que o perceptron traça uma **fronteira linear** entre classes: se a soma ponderada ultrapassa o limite, ele ativa (1); caso contrário, não ativa (0).

Apesar de simples, o perceptron pode implementar **portas lógicas básicas** como **AND**, **OR** e **NOT**. No entanto, problemas mais complexos, como o **XOR**, **não podem ser resolvidos por um perceptron único**, exigindo redes multicamadas.


In [1]:
from typing import List

Vector = List[float]

def dot(v: Vector, w: Vector) -> float:
    """Produto escalar entre dois vetores."""
    return sum(vi * wi for vi, wi in zip(v, w))

def step_function(x: float) -> float:
    """
    Função degrau clássica do perceptron.
    
    Retorna:
        1.0 se x >= 0
        0.0 caso contrário
    """
    return 1.0 if x >= 0 else 0.0

def perceptron_output(weights: Vector, bias: float, x: Vector) -> float:
    """
    Calcula a saída de um perceptron.

    Args:
        weights (Vector): pesos do perceptron (w1, w2, ..., wn)
        bias (float): termo de ajuste (threshold)
        x (Vector): vetor de entradas (x1, x2, ..., xn)
    
    Returns:
        float: 1.0 se o perceptron "disparar", 0.0 caso contrário
    """
    # soma ponderada + bias
    total_input = dot(weights, x) + bias
    
    # aplicar função de ativação degrau
    return step_function(total_input)


In [2]:
# AND
weights = [2, 2]
bias = -3

for x in [[0,0], [0,1], [1,0], [1,1]]:
    print(f"{x} -> {perceptron_output(weights, bias, x)}")


[0, 0] -> 0.0
[0, 1] -> 0.0
[1, 0] -> 0.0
[1, 1] -> 1.0


In [3]:
# or
weights = [2, 2]
bias = -1

for x in [[0,0], [0,1], [1,0], [1,1]]:
    print(f"{x} -> {perceptron_output(weights, bias, x)}")


[0, 0] -> 0.0
[0, 1] -> 1.0
[1, 0] -> 1.0
[1, 1] -> 1.0


In [4]:
#NOT
weigths = [-2]
bias = 1

for x in [[0], [1]]:
    print(f"{x} -> {perceptron_output(weigths, bias, x)}")


[0] -> 1.0
[1] -> 0.0


No entanto, alguns problemas não podem ser resolvidos por um único perceptron. 

Por exemplo, o **XOR (OU exclusivo)**, que deve retornar 1 quando apenas uma das entradas é 1, **não pode ser implementado com um único neurônio**. Isso acontece porque o XOR **não é linearmente separável**, ou seja, não existe uma linha única que consiga separar os casos de saída 0 dos casos de saída 1.

Porém, assim como os neurônios naturais, os neurônios artificiais **ganham poder de resolver problemas complexos quando estão conectados em múltiplas camadas**. É dessa forma que surgem as **redes neurais multicamadas (MLPs)**, capazes de aprender padrões mais sofisticados e resolver problemas como o XOR.


# Redes Neurais Feed-Forward

As **Redes Feed-Forward** são uma idealização da topologia do cérebro, sendo formadas por **camadas discretas de neurônios conectados em sequência**.  

Em geral, uma rede feed-forward possui:

- **Camada de entrada**: recebe as entradas e as transmite sem alterá-las.  
- **Uma ou mais camadas ocultas**: recebem as saídas da camada anterior, realizam cálculos e transmitem os resultados para a próxima camada.  
- **Camada de saída**: produz as saídas finais do modelo.  

Como no perceptron, cada neurônio possui **pesos** correspondentes a cada uma de suas entradas e um **viés (bias)**.  
Para simplificar, podemos **inserir o viés no final do vetor de pesos** e atribuir a cada neurônio uma entrada de viés sempre igual a 1.  

Além disso, em vez de usar a função degrau (`step_function`), que é discreta e não diferenciável, usamos uma **aproximação suave**: a **função sigmoide**, definida como:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

A função sigmoide **mapa qualquer valor real para o intervalo (0,1)**, permitindo que os neurônios produzam saídas contínuas e diferenciáveis — o que é essencial para o treinamento via **gradiente descendente e backpropagation**.

Por que usar a **sigmoid** e não <i>step_funcion</i>? A rede neural precisa da realização de cálculos, para o seu treinamento, e por isso funções suaves como a **sigmoid** são melhores

In [5]:
import math

def sigmoid(t: float) -> float:
    """Função sigmoide."""
    return 1 / (1 + math.exp(-t))

In [6]:
def neuron_output(weights: Vector, inputs: Vector) -> float:
    # o weights inclui  termo de viés, as entradas incluem um 1
    return sigmoid(dot(weights, inputs))

Partindo dessa função, representamos um neurônio como um vetor de pesos, cujo comprimento é um a mais do que o seu número de entradas (devido ao peso de viés). Então, representamos a rede neural como uma lista de <i>camadas</i> (sem entrada) formadas individualmente por uma lista de neurônios

Ou seja, representaremos a rede neural como uma lista (camadas) de listas (neurônios) de vetores (pesos).


In [7]:

def feed_forward(neural_network: List[List[Vector]], input_vector: Vector) -> List[Vector]:
    """Alimenta o vetor de entrada na rede neural.
       Retorna as saídas de todas as camadas (não só da última).
    """
    outputs: List[Vector] = []

    for layer in neural_network:
        input_with_bias = input_vector + [1]  # adicionar o viés
        output = [neuron_output(neuron, input_with_bias) for neuron in layer] # computa saída para cada neurônio
        outputs.append(output)

        #Entrada para a próxima camada é a saída da camada atual
        input_vector = output

    return outputs

Agora podemos criar a porta XOR que não conseguimos com um perceptron. Só precisamos ampliar os pesos para que os neuron_outputs fiquem muito próximos de 0 ou 1:

In [8]:
xor_network = [
    [ [20,20,-30], #and
      [20,20,-10] ], #or
      #camada de saída
    [ [-60,60,-30] ]  #neurônio de segunda entrada
]

assert 0.000 < feed_forward(xor_network, [0,0])[-1][0] < 0.001
assert 0.999 < feed_forward(xor_network, [0,1])[-1][0] < 1.001
assert 0.999 < feed_forward(xor_network, [1,0])[-1][0] < 1.001
assert 0.000 < feed_forward(xor_network, [1,1])[-1][0] < 0.001



# Retropropagação (Backpropagation)

Na prática, **não construímos redes neurais manualmente** (como no exemplo do XOR).  
Isso porque, em problemas reais — como reconhecimento de imagens — precisamos de centenas ou até milhares de neurônios, e não conseguimos definir manualmente os pesos de cada um deles.

Por isso, as redes neurais são **treinadas a partir de dados**.  
O algoritmo mais utilizado para isso é a **retropropagação (backpropagation)**, que aplica o **gradiente descendente** (ou suas variantes, como Adam, RMSProp etc.) para ajustar os pesos da rede.

---

## Intuição do algoritmo

Imagine um conjunto de treinamento formado por pares de **entrada** e **saída desejada**.  
Exemplo: o problema do **XOR**.

A rede começa com pesos aleatórios, e esses pesos vão sendo ajustados pelo seguinte processo:

1. **Feed-forward**  
   Executamos a passagem direta: a entrada percorre todas as camadas da rede até produzir uma saída.

2. **Cálculo do erro**  
   Como sabemos qual deveria ser a saída correta, calculamos a **função de perda**.  
   Exemplo: erro quadrático médio (MSE):  
   $$
   L = \frac{1}{2} \sum_i^n (y_i - \hat{y}_i)^2
   $$

3. **Gradiente na saída**  
   Computamos como a perda varia em relação aos pesos dos **neurônios de saída**.  
   (Aqui entra o cálculo de derivadas da função de ativação).

4. **Retropropagação dos erros**  
   Propagamos os gradientes de volta pela rede, ajustando também os pesos dos **neurônios ocultos**.

5. **Atualização dos pesos**  
   Ajustamos todos os pesos com um passo de **gradiente descendente**:  
   $$
   w \leftarrow w - \eta \frac{\partial L}{\partial w}
   $$  
   onde \(\eta\) é a **taxa de aprendizado**.

---

## Resumindo
A retropropagação é basicamente um ciclo:
- **propagar para frente** (feed-forward),  
- **calcular o erro**,  
- **propagar para trás** (backpropagation),  
- **ajustar os pesos**.  

Esse processo é repetido muitas vezes até que a rede aprenda a mapear corretamente as entradas para as saídas desejadas.


In [9]:
def sqerror_gradients(network: List[List[Vector]], input_vector: Vector, target_vector: Vector) -> List[List[Vector]]:
    """Quando houver uma rede neural, um vetor de entrada e um vetor de destino, faça uma previsão e compute o gradiente da perda doos erros quadráticos com relação aos pesos do neurônio.
    """

    #passe para frente
    hidden_outputs, outputs = feed_forward(network, input_vector)

    #gradientes associados às saídas de pré-ativação dos neurônios de saída
    output_deltas = [output * (1-output) * (output - target) for output, target in zip (outputs, target_vector)] # derivada da sigmoide vezes o erro

    #gradientes associados aos pesos dos neurônios de saída
    output_grads = [[output_deltas[i] * hidden_output for hidden_output in hidden_outputs + [1]] # adicionar o viés
                    for i, output_neuron in enumerate(network[-1])]
    
    #gradientes associados às saídas de pré-ativação dos neurônios ocultos
    hidden_deltas = [hidden_output * (1 - hidden_output) * dot(output_deltas, [n[i] for n in network[-1]]) # derivada da sigmoide vezes o erro
                     for i, hidden_output in enumerate(hidden_outputs)]
    
    #gradientes associados aos pesos dos neurônios ocultos
    hidden_grads = [[hidden_deltas[i] * input for input in input_vector + [1]] # adicionar o viés
                    for i, hidden_neuron in enumerate(network[0])]
    
    return [hidden_grads, output_grads]

Hora de treinar a nossa rede neural para aprender o XOR!

In [10]:
import random
random.seed(0)

#dados de treinamento para o XOR
xs = [[0,0], [0,1], [1,0], [1,1]]
ys = [[0], [1], [1], [0]]  #saídas desejadas

#comece com pesos aleatórios
network = [ #camada oculta: 2 entradas -> 2 saídas
            [[random.random() for _ in range(2+1)],
             [random.random() for _ in range(2+1)]],
              #camada de saída: 2 entradas -> 1 saída
            [[random.random() for _ in range(2+1)]]   
]

In [11]:
def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
    """Dá um passo de gradiente"""
    assert len(v) == len(gradient)
    return [v_i - step_size * grad_i for v_i, grad_i in zip(v, gradient)]

In [12]:
%pip install tqdm


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import tqdm

# Dados do XOR
xs = [[0,0],[0,1],[1,0],[1,1]]
ys = [[0],[1],[1],[0]]

# Rede inicial: 2 neurônios na camada oculta, 1 na de saída
import random
random.seed(0)

network = [
    [ [random.uniform(-1,1) for _ in range(3)],  # neurônio 1 (oculta)
      [random.uniform(-1,1) for _ in range(3)] ],# neurônio 2 (oculta)
    [ [random.uniform(-1,1) for _ in range(3)] ] # saída
]

learning_rate = 1.0

for epoch in tqdm.tqdm(range(20000), desc="Treinando XOR"):
    for x, y in zip(xs, ys):
        grads = sqerror_gradients(network, x, y)
        network = [[gradient_step(neuron, grad, learning_rate)
                   for neuron, grad in zip(layer, layer_grads)]
                   for layer, layer_grads in zip(network, grads)]

# Testes
assert feed_forward(network,[0,0])[-1][0] < 0.01
assert feed_forward(network,[0,1])[-1][0] > 0.99
assert feed_forward(network,[1,0])[-1][0] > 0.99
assert feed_forward(network,[1,1])[-1][0] < 0.01


Treinando XOR: 100%|██████████| 20000/20000 [00:00<00:00, 23732.39it/s]


In [14]:
print("Pesos da rede neural treinada:\n")

for i, layer in enumerate(network):
    print(f"Camada {i+1}:")
    for j, neuron in enumerate(layer):
        print(f"  Neurônio {j+1}: {neuron}")
    print()

for x in [[0,0],[0,1],[1,0],[1,1]]:
    y = feed_forward(network, x)[-1][0]
    print(f"{x} -> {y:.4f}")



Pesos da rede neural treinada:

Camada 1:
  Neurônio 1: [-5.281163769910824, -5.297755324303967, 7.891906536046263]
  Neurônio 2: [-6.814847848206161, -6.8995661206393795, 2.8780398608497513]

Camada 2:
  Neurônio 1: [11.207481902173496, -11.336295008052474, -5.367409380565893]

[0, 0] -> 0.0074
[0, 1] -> 0.9923
[1, 0] -> 0.9923
[1, 1] -> 0.0094
